In [ ]:
import mlrun
import os
%config Completer.use_jedi = False

In [ ]:
project = mlrun.load_project(name="load-project", url="git://github.com/amit-elbaz/load_project.git", context="./context",clone=True, user_project=True)

In [ ]:
print(project.to_yaml())

### Regular jobs

In [ ]:
for i in range(3):
    job_num = str(i)
    
    # Every 30 min
    project.run_function("normal_job_sec", name=f"sched_job_sec_30m_{job_num}", schedule="30 * * * *", watch=False)
    project.run_function("normal_job_min", name=f"sched_job_min_30m_{job_num}", schedule="30 * * * *", watch=False)
    project.run_function("normal_job_hour", name=f"sched_job_hour_30m_{job_num}", schedule="30 * * * *", watch=False)
    
    # Every 1 hour
    project.run_function("normal_job_sec", name=f"sched_job_sec_1h_{job_num}", schedule="0 * * * *", watch=False)
    project.run_function("normal_job_min", name=f"sched_job_min_1h_{job_num}", schedule="0 * * * *", watch=False)
    project.run_function("normal_job_hour", name=f"sched_job_hour_1h_{job_num}", schedule="0 * * * *", watch=False)

    # Every 4 hours
    project.run_function("normal_job_sec", name=f"sched_job_sec_4h_{job_num}", schedule="0 */4 * * *", watch=False)
    project.run_function("normal_job_min", name=f"sched_job_min_4h_{job_num}", schedule="0 */4 * * *", watch=False)
    project.run_function("normal_job_hour", name=f"sched_job_hour_4h_{job_num}", schedule="0 */4 * * *", watch=False)
    
    # Every 24 hours
    project.run_function("normal_job_sec", name=f"sched_job_sec_24h_{job_num}", schedule="0 0 */1 * *", watch=False)
    project.run_function("normal_job_min", name=f"sched_job_min_24h_{job_num}", schedule="0 0 */1 * *", watch=False)
    project.run_function("normal_job_hour", name=f"sched_job_hour_24h_{job_num}", schedule="0 0 */1 * *", watch=False)

    # Every 3 days
    project.run_function("normal_job_sec", name=f"sched_job_sec_3d_{job_num}", schedule="0 0 */3 * *", watch=False)
    project.run_function("normal_job_min", name=f"sched_job_min_3d_{job_num}", schedule="0 0 */3 * *", watch=False)
    project.run_function("normal_job_hour", name=f"sched_job_hour_3d_{job_num}", schedule="0 0 */3 * *", watch=False)

### Spark

In [ ]:
spark_func = mlrun.code_to_function(name="spark-read",
                                    kind="spark",
                                    handler="spark_handler",
                                    filename="spark_jobs_func.py", requirements=["scikit-learn"]
                                   ).apply(mlrun.auto_mount())
spark_func.with_executor_requests(cpu="1",mem="1G")
spark_func.with_driver_requests(cpu="1",mem="1G")
spark_func.with_driver_limits(cpu="1")
spark_func.with_executor_limits(cpu="1")
spark_func.with_igz_spark()
spark_func.spec.image_pull_policy = "Always"
spark_func.spec.replicas = 2

In [ ]:
spark_func.deploy()

In [ ]:
spark_func.run(schedule="0 * * * *")

### Dask

In [ ]:
dask_cluster = mlrun.new_function("dask-cluster", kind='dask', image='mlrun/ml-models')
dask_cluster.apply(mlrun.mount_v3io())        # add volume mounts
dask_cluster.spec.service_type = "NodePort"   # open interface to the dask UI dashboard
dask_cluster.spec.replicas = 1             # define one container
dask_cluster.set_env("MLRUN_DBPATH",os.environ["MLRUN_DBPATH"])
dask_cluster.set_env("MLRUN_DEFAULT_PROJECT",project.name)
uri = dask_cluster.save()

In [ ]:
project.run_function('dask_func',hyperparams={"i":[1,10,20,30,40]},schedule="0 * * * *", hyper_param_options={"strategy":"list","parallel_runs":1,"dask_cluster_uri":uri,"teardown_dask":True})

### Nuclio

In [ ]:
for i in range(3):
    job_num = str(i)
    # Create a simple nuclio function
    project.deploy_function("nuclio_func", tag=job_num)


### Serving

In [ ]:
models_dir = mlrun.get_sample_path("models/serving/")

suffix = (
    mlrun.__version__.split("-")[0].replace(".", "_")
    if sys.version_info[1] > 7
    else "3.7"
)


framework_sklearn = "sklearn"  # change to 'keras' to try the 2nd option
kwargs = {}

serving_class = "mlrun.frameworks.sklearn.SklearnModelServer"
model_path = models_dir + f"sklearn-{suffix}.pkl"
image = "mlrun/mlrun"

model_object = project.log_model(f"{framework_sklearn}-model", model_file=model_path, **kwargs)
serving_sklearn = mlrun.new_function("serving_sklearn", image=image, kind="serving", requirements=["scikit-learn"])
serving_sklearn.add_model(
    framework_sklearn, model_path=model_object.uri, class_name=serving_class, to_list=True
)
project.deploy_function(serving_sklearn)


framework_keras = "keras"
serving_class = "mlrun.frameworks.tf_keras.TFKerasModelServer"
model_path = models_dir + "keras.h5"
image = "mlrun/ml-models"  # or mlrun/ml-models-gpu when using GPUs
kwargs["labels"] = {"model-format": "h5"}

model_object = project.log_model(f"{framework_keras}-model", model_file=model_path, **kwargs)
serving_tensorf = mlrun.new_function("serving_tensorf", image=image, kind="serving", requirements=["tensorflow"])
serving_tensorf.add_model(
    framework_keras, model_path=model_object.uri, class_name=serving_class, to_list=True
)

project.deploy_function(serving_tensorf)

## Workflow

In [ ]:
project.run(workflow_path='workflow.py',watch=True, schedule="0 * * * *")